***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-005b  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 10 12:21 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [5]:
spark

In [6]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_ir_os' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and "multistream" in b.name and ".parquet" in b.name:
        paths.append(full_path+b.name)
        print(full_path+b.name)
#     if b.name != 'graphframes.sh':
#         paths.append(full_path+b.name)

gs://project_ir_os/multistream10_preprocessed.parquet
gs://project_ir_os/multistream11_part2_preprocessed.parquet
gs://project_ir_os/multistream11_preprocessed.parquet
gs://project_ir_os/multistream12_part2_preprocessed.parquet
gs://project_ir_os/multistream12_preprocessed.parquet
gs://project_ir_os/multistream13_part2_preprocessed.parquet
gs://project_ir_os/multistream13_preprocessed.parquet
gs://project_ir_os/multistream14_part2_preprocessed.parquet
gs://project_ir_os/multistream14_preprocessed.parquet
gs://project_ir_os/multistream15_part2_preprocessed.parquet
gs://project_ir_os/multistream15_part3_preprocessed.parquet
gs://project_ir_os/multistream15_preprocessed.parquet
gs://project_ir_os/multistream16_part2_preprocessed.parquet
gs://project_ir_os/multistream16_part3_preprocessed.parquet
gs://project_ir_os/multistream16_preprocessed.parquet
gs://project_ir_os/multistream17_part2_preprocessed.parquet
gs://project_ir_os/multistream17_part3_preprocessed.parquet
gs://project_ir_os/mul

# Building an inverted index for title

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [15]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [8]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp_title.py

ls: cannot access 'inverted_index_gcp.py': No such file or directory


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp_title.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp_title import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [16]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

###################### word_count ######################

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE

  word_counter = Counter(tokens)
  # Work 1 - Bad Time
  # ret_list = []
  # for token in tokens:
  #   if (token not in all_stopwords) and ((token, (id, word_counter.get(token))) not in ret_list):
  #     ret_list.append((token, (id, word_counter.get(token))))
  # return ret_list

  # Work 2 - Best time
  return ([(k, (id, v)) for k, v in word_counter.items() if k not in all_stopwords and v > 0 ])


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
###################### doc_normaliztion_count ######################
import math

def doc_normaliztion_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  docs_tokens = [i for i in tokens if not i in all_stopwords] # query without stop words
  word_counter = Counter(docs_tokens)

  # calculate (1/|d|)
  sum = 0
  for term in word_counter:
    sum += (word_counter[term] ** 2)
  if math.sqrt(sum) == 0:
    nf = 1
  else:
    nf = (1 / math.sqrt(sum))

  return (id,nf)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


###################### reduce_word_counts ######################

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE

  return sorted(unsorted_pl, key=lambda x: x[0])


###################### calculate_df ######################

import enum
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  #print(postings.map(lambda x: x[1]).take(1))

  # Work 1 - Two lines
  # RDD_dict = postings.collectAsMap()
  # return sc.parallelize([(token, len(RDD_dict[token])) for token in RDD_dict])

  # Work 2 - One line!!!
  return postings.mapValues(lambda df: len(df))


###################### partition_postings_and_write  ######################

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE

  # work 1 - 2 lines
  # RDD_dict = postings.map(lambda token: (token2bucket_id(token[0]), token)).groupByKey()
  # #print(RDD_dict.take(1))
  # return RDD_dict.map(InvertedIndex.write_a_posting_list)

  # Work 2 - one line!!!
  return postings.map(lambda token: (token2bucket_id(token[0]), token)).groupByKey().map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))


In [17]:
docs_title_counts = doc_title_pairs.map(lambda x: doc_normaliztion_count(x[0], x[1])) # rdd -> normalize docs

# collectAsMap collects the results to the master node's memory as a dictionary.
# we know it's not so big so this is okay.
nf_dict = docs_title_counts.collectAsMap()


doc_title_pairs1 = parquetFile.select("id", "title").rdd
title_dict = doc_title_pairs1.collectAsMap()

In [18]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
# postings_filtered = postings.filter(lambda x: len(x[1])>50) # TODO: remove for title and anchor
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()
index_const_time = time() - t_start

In [19]:
print(index_const_time)

35.882874727249146


In [20]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [21]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
inverted.nf = nf_dict
inverted.numberOfDocs = len(nf_dict)
inverted.TitlesOfDocs = title_dict
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# write the global stats out
inverted.write_index('.', 'index_title') # TODO::
# upload to gs
index_src = "index_title.pkl" #TODO::
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}' # TODO::
!gsutil cp $index_src $index_dst

Copying file://index_title.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][321.2 MiB/321.2 MiB]                                                
Operation completed over 1 objects/321.2 MiB.                                    


In [22]:
!gsutil ls -lh $index_dst

321.16 MiB  2023-01-10T12:31:50Z  gs://project_ir_os/postings_gcp_title/index_title.pkl
TOTAL: 1 objects, 336761357 bytes (321.16 MiB)


# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp_body/'
!gsutil du -sh "$index_dst"

5.93 GiB     gs://sise_ir_assignment3/postings_gcp
